In [7]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import math
import datetime as dt
import time

In [8]:
import ccxt 
import pprint

In [33]:
## 권한설정
bitmex_api_key = 'klXJROefQTovG_9cidLgFE3-' 
bitmex_api_secret = '2CuRU6p1xzGDy8Bh5YaOtnALuhky7aqIw8OSpZow96Pv4jyk'

btmx  = ccxt.bitmex({
    'enableRateLimit': True,
    'apiKey': bitmex_api_key,
    'secret': bitmex_api_secret,
})
btmx.verbose = True

In [23]:
def get_price(ticker):
    btmx = ccxt.bitmex(config={
        'apiKey': bitmex_api_key, 
        'secret': bitmex_api_secret,
        'enableRateLimit': True,
        'options': {
            'defaultType': 'future'
        }
    })
    
    price = btmx.fetch_ticker(ticker)
    
    return price['close']

In [24]:
## 이전 4개분기에서 도출
cri_ustd = 0.013286910557086425
cri_lstd = 0.0013898117100966975

In [27]:
temp_ls = []
pos = 'neut'
while True:
    price_near = get_price('XBTH22')
    price_longer = get_price('XBTM22')
    spread_ratio = (price_longer - price_near)/price_near
    ## Signal => Long(원분기물 Long, 근분기물 Short)
    prev_pos = pos
    if spread_ratio < cri_lstd:
        if pos == 'long':
            print(f'keep position: {pos}')
        elif pos == 'short':
            pos = 'long'
            print(f'reverse position: short->{pos}')
            ## LSS&NLS(S포지션 청산),  LLB & NSB(L포지션 진입)
        else: ## neut
            pos = 'long'
            ## LLB&NSB
            print(f'new position: neut->{pos}')
    
    ## Signal => Short(원분기물 Short, 근분기물 Long)
    elif spread_ratio > cri_ustd:
        if pos == 'long':
            pos = 'short'
            print(f'reverse position: long->{pos}')
            ## LLS&NSS(L포지션 청산),  LSB & NLB(S포지션 진입)
        elif pos == 'short':
            print(f'keep position: {pos}')
        else: ## neut
            pos = 'short'
            ## LSB & NLB(S포지션 진입)
            print(f'new position: neut->{pos}')
    
    ## Signal => neut(청산)
    else:
        if pos == 'long':
            pos = 'neut' 
            print(f'new position: long->{pos}')
            ## LLS&NSS(L포지션 청산)
        elif pos == 'short':
            pos = 'neut'
            print(f'new position: short->{pos}')
            ## LSS&NLS(S포지션 청산)
        else: ## neut
            pos = 'neut'
            print(f'keep position: {pos}')
    temp_ls.append([price_near, price_longer, prev_pos, pos])
    time.sleep(30)

keep position: neut
keep position: neut


KeyboardInterrupt: 

In [56]:
btmx.markets['BTC/USD:BTC-220325']

{'id': 'XBTH22',
 'symbol': 'BTC/USD:BTC-220325',
 'base': 'BTC',
 'quote': 'USD',
 'baseId': 'XBT',
 'quoteId': 'USD',
 'active': True,
 'type': 'future',
 'linear': False,
 'inverse': True,
 'spot': False,
 'swap': False,
 'future': True,
 'option': False,
 'margin': False,
 'contract': True,
 'contractSize': -100000000.0,
 'expiry': 1648209600000,
 'expiryDatetime': '2022-03-25T12:00:00.000Z',
 'optionType': None,
 'strike': None,
 'settle': 'BTC',
 'settleId': 'XBt',
 'precision': {'amount': 100.0, 'price': 0.5},
 'limits': {'leverage': {'min': 1.0, 'max': 100.0},
  'amount': {'min': None, 'max': None},
  'price': {'min': None, 'max': 1000000.0},
  'cost': {'min': None, 'max': 10000000.0}},
 'info': {'symbol': 'XBTH22',
  'rootSymbol': 'XBT',
  'state': 'Open',
  'typ': 'FFCCSX',
  'listing': '2021-09-15T04:00:00.000Z',
  'front': '2022-01-28T12:00:00.000Z',
  'expiry': '2022-03-25T12:00:00.000Z',
  'settle': '2022-03-25T12:00:00.000Z',
  'listedSettle': '2022-03-25T12:00:00.000Z',

In [65]:
market_order = btmx.create_order("XBTH22", "market","buy", 100, {'leverage': 3})


fetch Request: bitmex POST https://www.bitmex.com/api/v1/order RequestHeaders: {'Content-Type': 'application/json', 'api-key': 'klXJROefQTovG_9cidLgFE3-', 'api-expires': '1648110542', 'api-signature': '5a0ee7391e427966794b9502e2eb5c882e073286b9a7409e4abd0f001a0a88b5', 'Accept-Encoding': 'gzip, deflate'} RequestBody: {"symbol":"XBTH22","side":"Buy","orderQty":100.0,"ordType":"Market"}

fetch Response: bitmex POST https://www.bitmex.com/api/v1/order 400 ResponseHeaders: {'Date': 'Thu, 24 Mar 2022 08:28:58 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Content-Length': '127', 'Connection': 'keep-alive', 'Set-Cookie': 'AWSALBTG=ECf6NhRZtn+sh+bLkYsiyuO2XCyAgL/2hbMtmqcPsaEhcom3WscCFUDpQEpjQLPAf6KLUWZgu8Z35PLDHqKKpLgCCA3DTDPA+br+dNhZNOjnZrZDszFl1d9dpRyX6OjInkvAGLGY/kzbVQMaokkA6TC/2Dqr+u0Djq4NDSCHI0ZW; Expires=Thu, 31 Mar 2022 08:28:58 GMT; Path=/, AWSALBTGCORS=ECf6NhRZtn+sh+bLkYsiyuO2XCyAgL/2hbMtmqcPsaEhcom3WscCFUDpQEpjQLPAf6KLUWZgu8Z35PLDHqKKpLgCCA3DTDPA+br+dNhZNOjnZrZDszFl1d9dpR

InsufficientFunds: bitmex {"error":{"message":"Account has insufficient Available Balance, 2557 XBt required","name":"CodedHTTPError","details":"19000"}}

In [31]:
order_long = btmx.create_market_buy_order(symbol="XBTH22", amount=0)

BadRequest: bitmex {"error":{"message":"Invalid orderQty","name":"ValidationError"}}

In [103]:
test = pd.DataFrame(temp_ls, columns=['near', 'longer', 'prev_pos', 'pos'])
test['spread'] = test['longer'] - test['near']

In [104]:
test['earn'] = test['spread'] - test['spread'].shift(1)
test['fee'] = (test['near'] + test['longer']) * 0.0004


test['pos_earn'] = test['earn'].copy()
test['pos_earn'][test['prev_pos'] == 'neut'] = 0
test['pos_earn'][test['prev_pos'] == 'short'] = -test['pos_earn'][test['prev_pos'] == 'short']

test['pos_fee'] = test['fee'].copy()
test['pos_fee'][test['prev_pos'] == test['pos']] = 0

test = test.dropna()

In [105]:
test['pos_earn'].sum()

35.400000000001455

In [106]:
test['pos_fee'].sum()

101.71712000000002

In [107]:
test[test['pos_fee'] != 0]

,near,longer,prev_pos,pos,spread,earn,fee,pos_earn,pos_fee
17,42080.1,42619.1,short,neut,539.0,-45.5,33.87968,45.5,33.87968
22,42125.4,42688.9,neut,short,563.5,19.7,33.92572,0.0,33.92572
23,42114.3,42665.0,short,neut,550.7,-12.8,33.91172,12.8,33.91172


In [45]:
if spread_ratio > cri_ustd:
    print('short')
    ## 원분기물 Short, 근분기물 Long
    order_long = binance.create_market_buy_order(
    symbol="BTCUSDT_220325",
    amount=0.001)
    order_short = binance.create_market_sell_order(
    symbol="BTCUSDT_220624",
    amount=0.001)
    
elif spread_ratio < cri_lstd:
    print('long')
    ## 원분기물 Long, 근분기물 Short
    order_long = binance.create_market_buy_order(
    symbol="BTCUSDT_220624",
    amount=0.001)
    order_short = binance.create_market_sell_order(
    symbol="BTCUSDT_220325",
    amount=0.001)
    
else:
    print('neut')

short


0.03701178837101567

In [ ]:


btc = binance.fetch_ticker("BTC/USDT")
print(btc